# Introducción al Notebook

#### Desarrollado por Sergio Mahía INSO 3A

Este notebook aborda la generación de texto utilizando Redes Neuronales Recurrentes (RNN) para imitar las intervenciones de tres parlamentarios españoles: Sánchez, Casado y Abascal. El objetivo es usar modelos RNN ya construidos para cada uno y generar nuevas intervenciones en su estilo.

#### Estructura del Notebook:

1. **Importación y Configuración**: Se importan las bibliotecas necesarias y se configura el entorno para entrenar y evaluar los modelos.

2. **Mapeo de Caracteres**: Se crean los diccionarios de conversión entre caracteres e índices para procesar el texto en forma numérica.

3. **Construcción de Modelos RNN**: Se crean modelos LSTM para cada parlamentario, que aprenden dependencias lingüísticas complejas.

4. **Carga de Pesos**: Se cargan los modelos preentrenados para generar texto con cada uno.

5. **Generación de Texto**: Se define una función general para generar texto a partir de una cadena inicial, controlando la creatividad con el parámetro de `temperature`.

6. **Simulación del Diálogo**: Los modelos generan intervenciones alternadas para simular un diálogo entre los parlamentarios, utilizando la última frase generada como la semilla del siguiente modelo.

Este proyecto ofrece una visión interesante de cómo las RNN pueden captar patrones lingüísticos de figuras públicas y generar nuevas intervenciones con el estilo aprendido.


### Importación de Bibliotecas Necesarias

En esta celda se importan las bibliotecas esenciales para construir y entrenar modelos de redes neuronales recurrentes (RNN) con TensorFlow y Keras. A continuación, se detallan los elementos importados:

- **TensorFlow**: Framework utilizado para crear y entrenar redes neuronales. Aquí lo importamos como `tf`.
- **Keras**: Biblioteca de alto nivel sobre TensorFlow que facilita la construcción de modelos de redes neuronales. Importamos:
  - `Sequential`: Para definir un modelo secuencial, donde las capas se añaden una tras otra, ideal para la estructura de nuestra RNN.
  - `Embedding`, `LSTM`, `Dense`, `Dropout`: Capas que serán utilizadas para construir y entrenar la RNN

Estas bibliotecas forman la base para construir y entrenar el modelo RNN que se encargará de aprender las intervenciones de los parlamentarios y generar texto basado en ellas.


In [1]:
# Importar TensorFlow, la biblioteca principal para la construcción de modelos de Deep Learning
import tensorflow as tf

# Importar clases de Keras necesarias para construir la arquitectura del modelo
from tensorflow.keras import Sequential  # Para construir modelos secuenciales, donde las capas se añaden en orden
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout  # Capas esenciales para construir una RNN

# Comprobar las versiones de TensorFlow y NumPy
print("Versión de TensorFlow:", tf.__version__)

Versión de TensorFlow: 2.17.1


### Construcción del Mapeo de Caracteres

En estas celdas se construyen los mapeos de caracteres a índices y viceversa a partir de un archivo de texto que contiene las intervenciones de cada parlamentario. Este mapeo es esencial para convertir el texto en una representación numérica que el modelo pueda procesar, ya que las redes neuronales trabajan con números y no directamente con caracteres.

- Primero se lee el archivo de texto completo y se convierte a una cadena usando la codificación `UTF-8`.
- Se extrae el conjunto de caracteres únicos en el texto para formar el vocabulario.
- Se construyen dos diccionarios: uno para convertir de carácter a índice (`char2idx[politico]`) y otro para convertir de índice a carácter (`idx2char[politico]`). Estos diccionarios serán utilizados durante el entrenamiento y la generación de texto para traducir entre caracteres y su representación numérica.

In [2]:
# Leer el archivo de texto con las intervenciones de Sánchez
with open("/content/intervencionesSanchez.txt", 'rb') as f:
    text = f.read().decode(encoding='utf-8')  # Leer el contenido del archivo y decodificarlo como UTF-8

# Crear el vocabulario de caracteres únicos presentes en el texto
vocabSanchez = sorted(set(text)) # `sorted(set(text))` proporciona una lista ordenada de caracteres únicos

# Crear el diccionario para mapear cada carácter a un índice único
char2idxSanchez = {c: i for i, c in enumerate(vocabSanchez)} # Diccionario para convertir carácter a índice

# Crear el diccionario inverso para mapear cada índice a su carácter correspondiente
idx2charSanchez = {i: c for c, i in char2idxSanchez.items()} # Diccionario para convertir índice a carácter

In [3]:
# Leer el archivo de texto con las intervenciones de Casado
with open("/content/intervencionesCasado.txt", 'rb') as f:
    text = f.read().decode(encoding='utf-8')  # Leer y decodificar el archivo como UTF-8

# Crear el vocabulario de caracteres únicos presentes en el texto
vocabCasado = sorted(set(text)) # `sorted(set(text))` proporciona una lista ordenada de caracteres únicos

# Crear el diccionario para mapear cada carácter a un índice único
char2idxCasado = {c: i for i, c in enumerate(vocabCasado)} # Diccionario para convertir carácter a índice

# Crear el diccionario inverso para mapear cada índice a su carácter correspondiente
idx2charCasado = {i: c for c, i in char2idxCasado.items()} # Diccionario para convertir índice a carácter

In [4]:
# Leer el archivo de texto con las intervenciones de Abascal
with open("/content/intervencionesAbascal.txt", 'rb') as f:
    text = f.read().decode(encoding='utf-8')

# Crear el vocabulario de caracteres únicos presentes en el texto
vocabAbascal = sorted(set(text)) # `sorted(set(text))` proporciona una lista ordenada de caracteres únicos


# Crear el diccionario para mapear cada carácter a un índice único
char2idxAbascal = {c: i for i, c in enumerate(vocabAbascal)} # Diccionario para convertir carácter a índice

# Crear el diccionario inverso para mapear cada índice a su carácter correspondiente
idx2charAbascal = {i: c for c, i in char2idxAbascal.items()} # Diccionario para convertir índice a carácter

### Construcción del Modelo RNN

En esta celda se define una función para construir un modelo de red neuronal recurrente (RNN) utilizando capas LSTM para la generación de texto. El modelo consta de varias capas, cada una con una función específica:

1. **Embedding Layer**: La primera capa es una capa de `Embedding`, que convierte los índices numéricos (representación de caracteres) en vectores densos de una dimensión fija (`embedding_dim`). Esto permite representar el significado de cada carácter en un espacio vectorial.

2. **Primera Capa LSTM**: La primera capa `LSTM` se encarga de capturar las dependencias temporales largas en el texto, permitiendo al modelo aprender contextos a largo plazo. Se utiliza `stateful=True` para mantener el estado de la red entre lotes de entrenamiento.

3. **Dropout**: Añadimos una capa `Dropout` con una probabilidad del 40% para evitar el sobreajuste, apagando de forma aleatoria algunas neuronas durante el entrenamiento.

4. **Segunda Capa LSTM**: La segunda capa `LSTM` refina los patrones aprendidos por la primera capa, utilizando la mitad de las unidades (`rnn_units // 2`) para reducir la dimensionalidad y evitar el sobreajuste.

5. **Dense Intermedia y Dropout**: Una capa `Dense` con 512 neuronas y activación `ReLU` proporciona capacidad adicional al modelo para aprender patrones no lineales. Se añade `Dropout` para prevenir el sobreajuste.

6. **Capa de Salida**: Finalmente, una capa `Dense` con un tamaño de salida igual al vocabulario (`vocab_size`) devuelve una probabilidad para cada carácter en el vocabulario.

Este modelo está diseñado para aprender secuencias de texto y generar caracteres basados en contextos previos. Las dos capas LSTM ayudan a que el modelo capture relaciones complejas entre los caracteres, y las capas de Dropout ayudan a regularizar el modelo.


In [5]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential()  # Crear un modelo secuencial para apilar las capas una tras otra

    # Capa de tipo embedding: convierte los índices numéricos de los caracteres en vectores densos
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))

    # Primera capa LSTM: captura las dependencias temporales largas del texto de entrada
    model.add(LSTM(
        rnn_units,  # Número de unidades LSTM para capturar dependencias complejas
        return_sequences=True,  # Devolver secuencias completas para la siguiente capa LSTM
        stateful=True,  # Mantener el estado entre los lotes de entrenamiento, útil para generación de texto
        recurrent_initializer='glorot_uniform'  # Inicializar los pesos de manera uniforme
    ))
    model.add(Dropout(0.4))  # Regularización para evitar sobreajuste apagando neuronas aleatoriamente

    # Segunda capa LSTM: refina los patrones aprendidos en la primera capa
    model.add(LSTM(
        rnn_units // 2,  # Mitad de unidades para reducir la complejidad y prevenir el sobreajuste
        return_sequences=True,  # Continuar devolviendo secuencias para la siguiente capa
        stateful=True,  # Mantener el estado entre lotes
        recurrent_initializer='glorot_uniform'
    ))
    model.add(Dropout(0.4))  # Añadir Dropout para mayor regularización

    # Capa Dense intermedia: proporciona capacidad adicional de representación no lineal
    model.add(Dense(512, activation="relu"))  # 512 neuronas con activación ReLU para más capacidad expresiva
    model.add(Dropout(0.4))  # Regularización adicional

    # Capa de salida: devuelve una probabilidad para cada carácter en el vocabulario
    model.add(Dense(vocab_size))  # Número de neuronas igual al tamaño del vocabulario para predicción de cada carácter

    return model  # Devolver el modelo construido

### Definición de Hiperparámetros del Modelo

En esta celda se definen dos hiperparámetros importantes para la construcción del modelo RNN:

- **`embedding_dim = 256`**: Define la dimensión de los vectores de embedding. Este valor representa el tamaño del espacio vectorial donde se proyectarán los caracteres del vocabulario. Un mayor valor de `embedding_dim` puede proporcionar una mejor representación del texto, ya que cada carácter tendrá una representación más rica, aunque también aumenta la complejidad computacional.

- **`rnn_units = 1024`**: Define el número de unidades en la primera capa LSTM. Este valor determina la capacidad de la red para aprender patrones complejos en las secuencias de texto. Un valor alto, como `1024`, le permite a la red aprender una variedad de dependencias temporales, aunque requiere más recursos computacionales para entrenar.

In [6]:
embedding_dim = 256  # Tamaño de los vectores de embedding, que determina la representación de los caracteres
rnn_units = 1024  # Número de unidades en la primera capa LSTM, para capturar dependencias complejas en el texto

### Definición de la Forma de Entrada del Modelo

En esta celda se define el **`input_shape`** del modelo, que representa la forma de los datos de entrada que recibirá la red. En este caso:

- **`input_shape = (1, None)`**: Se indica que la entrada del modelo tendrá una secuencia de longitud variable (`None`), lo cual es ideal para trabajar con texto donde las secuencias pueden variar en tamaño. El `1` indica que solo se procesará una muestra por vez, que es típico en modelos `stateful` donde se quiere mantener el estado entre predicciones consecutivas.

In [7]:
input_shape = (1, None)  # Definición de la forma de entrada: un lote con secuencia de longitud variable

### Construcción y Carga de los Modelos para Generación de Texto

En esta sección se construyen y cargan los pesos para los modelos RNN correspondientes a los tres parlamentarios: Sánchez, Casado y Abascal. Cada modelo se define utilizando la función `build_model`, se ajusta con el método `build` según la forma de entrada especificada, y luego se cargan los pesos preentrenados desde archivos guardados previamente.

- **Construcción del Modelo**: Para cada parlamentario se llama a la función `build_model`, especificando el tamaño del vocabulario respectivo, la dimensión de embedding y las unidades de la LSTM.
- **Ajuste de la Forma de Entrada**: Con el método `build`, se especifica la forma de entrada para asegurar que el modelo esté preparado para procesar secuencias de longitud variable, con un tamaño de lote de `1` para la generación de texto.
- **Carga de Pesos Preentrenados**: Los pesos se cargan desde archivos `.keras` previamente guardados, permitiendo al modelo utilizar los parámetros aprendidos durante el entrenamiento.

Cada modelo está ahora listo para ser utilizado en la generación de texto, utilizando el conocimiento específico de las intervenciones parlamentarias de cada uno.

In [8]:
# Construcción y carga del modelo para las intervenciones de Sánchez
model_sanchez = build_model(len(vocabSanchez), embedding_dim, rnn_units, batch_size=1)  # Crear el modelo para Sánchez
model_sanchez.build(input_shape=input_shape)  # Ajustar la forma de entrada para secuencias de longitud variable
model_sanchez.load_weights("/content/model_sanchez_150_2024.keras")  # Cargar los pesos preentrenados del modelo

In [9]:
# Construcción y carga del modelo para las intervenciones de Casado
model_casado = build_model(len(vocabCasado), embedding_dim, rnn_units, batch_size=1)  # Crear el modelo para Casado
model_casado.build(input_shape=input_shape)  # Ajustar la forma de entrada para secuencias de longitud variable
model_casado.load_weights("/content/model_casado_150_2024.keras")  # Cargar los pesos preentrenados del modelo

In [10]:
# Construcción y carga del modelo para las intervenciones de Abascal
model_abascal = build_model(len(vocabAbascal), embedding_dim, rnn_units, batch_size=1)  # Crear el modelo para Abascal
model_abascal.build(input_shape=input_shape)  # Ajustar la forma de entrada para secuencias de longitud variable
model_abascal.load_weights("/content/model_abascal_150_2024.keras")  # Cargar los pesos preentrenados del modelo

### Función Generalizada para Generar Texto

En esta celda se define una función generalizada para generar texto a partir de cualquier modelo RNN entrenado. Esta función permite evitar la duplicación de código al ser reutilizable para diferentes parlamentarios con sus vocabularios y modelos correspondientes.

- **`generate_text(model, start_string, char2idx, idx2char, temperature=0.2, num_generate=1000)`**:
  - **`model`**: Modelo RNN entrenado que se utilizará para generar el texto.
  - **`start_string`**: Cadena inicial a partir de la cual se generará el texto. Este texto actúa como semilla para la generación.
  - **`char2idx`** y **`idx2char`**: Diccionarios de mapeo de carácter a índice y de índice a carácter, utilizados para traducir entre caracteres y sus representaciones numéricas.
  - **`temperature`**: Controla la aleatoriedad del texto generado. Valores bajos (e.g., `0.2`) resultan en predicciones más predecibles, mientras que valores más altos (`0.7`) introducen más diversidad y creatividad.
  - **`num_generate`**: Número de caracteres a generar.

La lógica de la función es la siguiente:
1. **Conversión de la Cadena Inicial**: Convierte la cadena inicial a una lista de índices (`input_eval`) para que el modelo pueda procesarla.
2. **Generación del Texto**:
   - Se realiza una predicción a partir de la entrada actual (`input_eval`) utilizando el modelo.
   - La predicción se ajusta según la `temperature` para controlar la creatividad de la salida.
   - Se selecciona el siguiente carácter utilizando una distribución categórica, que permite generar caracteres según las probabilidades predichas.
   - La entrada se actualiza con el nuevo carácter generado, y el carácter correspondiente se añade a la lista `text_generated`.
3. **Devolución del Texto Completo**: Devuelve la cadena inicial concatenada con el texto generado, lo cual facilita ver cómo el modelo expande el contexto original.

Esta función se utiliza de manera consistente para los tres parlamentarios, lo que mejora la claridad y la eficiencia del código.


In [11]:
def generate_text(model, start_string, char2idx, idx2char, temperature=0.2, num_generate=1000):
    input_eval = [char2idx[s] for s in start_string if s in char2idx]  # Convertir la cadena inicial a índices
    input_eval = tf.expand_dims(input_eval, 0)  # Añadir una dimensión batch

    text_generated = []  # Lista para almacenar el texto generado

    for _ in range(num_generate):
        predictions = model(input_eval)  # Hacer una predicción usando el modelo
        predictions = tf.squeeze(predictions, 0)  # Eliminar la dimensión batch
        predictions = predictions / temperature  # Ajustar las probabilidades según la temperatura
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()  # Seleccionar un índice según la distribución categórica
        input_eval = tf.expand_dims([predicted_id], 0)  # Actualizar la entrada para el próximo carácter
        text_generated.append(idx2char[predicted_id])  # Añadir el carácter generado a la lista de salida

    return start_string + ''.join(text_generated)  # Devolver la cadena inicial junto con el texto generado

### Función para Extraer la Última Frase Generada

En esta celda se define una función para extraer la última frase completa generada a partir del texto:

- **`extract_last_sentence(text)`**:
  - **Parámetro `text`**: El texto completo generado por el modelo.
  - **Proceso**: El texto se divide en oraciones utilizando el carácter `.`,`!` o `?` como delimitador.
  - **Resultado**: Se devuelve la penúltima oración (`sentences[-2]`) para asegurar que sea una oración completa y no una parcial generada al final del texto. Se utiliza `strip()` para eliminar espacios en blanco. Si solo existe una oración, se devuelve todo el texto sin cambios.

Esta función permite extraer únicamente la última frase completa del texto generado, lo cual es útil para mantener coherencia en la salida y evitar fragmentos incompletos que puedan ocurrir al final.

In [12]:
import re

# Definir una función mejorada para extraer la última frase completa del texto generado
def extract_last_sentence(text):
    # Dividir el texto utilizando una expresión regular que considere ., ! y ? como delimitadores
    sentences = re.split(r'[.!?]', text)
    # Devolver la penúltima oración si existe, sino devolver todo el texto limpio de espacios
    return sentences[-2].strip() if len(sentences) > 1 else text.strip()

### Configuración Inicial para la Generación de Texto

En esta celda se define la configuración inicial para comenzar la generación de texto:

- **`seed_text = "El parlamento"`**: Texto inicial que se utilizará como entrada para los modelos RNN, actuando como la semilla de la generación.
- **`turns = 5`**: Número de turnos de conversación a generar. Esto se utilizará si los modelos "dialogan" entre sí, alternando respuestas durante múltiples turnos.

Esta configuración permite iniciar la generación con un contexto claro y definir la longitud del intercambio entre los modelos.

In [13]:
# Configuración inicial para la generación de texto
seed_text = "El parlamento"  # Texto inicial que servirá como punto de partida para la generación de texto
turns = 5  # Número de turnos de conversación que se desea generar

### Simulación del Diálogo entre Modelos

En esta celda se simula un diálogo entre los tres modelos entrenados para los parlamentarios (Sánchez, Casado y Abascal). Cada modelo genera texto a partir de la última frase generada por el modelo anterior, creando un intercambio continuo durante un número de turnos especificado.

- **Bucle de Diálogo**: Se ejecuta un bucle que itera el número de turnos definidos (`turns`).
  - **Generación de Texto**: En cada turno, cada modelo genera una respuesta a partir del texto inicial (`seed_text`) o la última frase generada.
  - **Extracción de la Última Frase**: La función `extract_last_sentence` se usa para extraer la última frase completa del texto generado, que luego se utiliza como entrada para el siguiente modelo.
  - **Temperatura Ajustada**: Se utiliza una temperatura de `0.4` para Sánchez y Casado (para mantener cierta coherencia) y `0.7` para Abascal, buscando mayor creatividad.

Este proceso simula cómo podrían interactuar los parlamentarios en un "diálogo", creando una secuencia fluida de respuestas.

In [14]:
# Simulación del diálogo entre los modelos de los tres parlamentarios
print("Inicio del diálogo:\n")

# Bucle para cada turno del diálogo, según el número de turnos definido
for i in range(turns):
    # Turno de Sánchez
    print(f"Turno {i+1} (Sánchez):")
    text_sanchez = generate_text(model_sanchez, seed_text, char2idxSanchez, idx2charSanchez, temperature=0.4)
    print(text_sanchez)
    seed_text = extract_last_sentence(text_sanchez)  # Extraer la última frase generada para usarla como entrada en el siguiente turno
    # Validar que seed_text no esté vacío
    if not seed_text:
        seed_text = "Por favor, continúe."  # Mensaje por defecto en caso de que `seed_text` sea vacío

    # Turno de Casado
    print(f"Turno {i+1} (Casado):")
    text_casado = generate_text(model_casado, seed_text, char2idxCasado, idx2charCasado, temperature=0.4)
    print(text_casado)
    seed_text = extract_last_sentence(text_casado)  # Extraer la última frase generada para usarla como entrada en el siguiente turno
    # Validar que seed_text no esté vacío
    if not seed_text:
        seed_text = "Por favor, continúe."  # Mensaje por defecto en caso de que `seed_text` sea vacío

    # Turno de Abascal
    print(f"Turno {i+1} (Abascal):")
    text_abascal = generate_text(model_abascal, seed_text, char2idxAbascal, idx2charAbascal, temperature=0.7)
    print(text_abascal)
    seed_text = extract_last_sentence(text_abascal)  # Extraer la última frase generada para usarla como entrada en el siguiente turno
    # Validar que seed_text no esté vacío
    if not seed_text:
        seed_text = "Por favor, continúe."  # Mensaje por defecto en caso de que `seed_text` sea vacío

    # Separación entre turnos para mayor claridad en la salida
    print("\n")

Inicio del diálogo:

Turno 1 (Sánchez):
El parlamento de los defensores de la Constitución española― vemos que los valores que tienen que ver con el estatuto del becario y también con reforzar todo lo que tiene que ver con la policía patriótica que ustedes pusieron en marcha cuando estaban gobernar, en ese sentido creo que también es un logro de este Gobierno, y por tanto de esta democracia, el que arrimar el hombro, tiene que ser útil, señoría. La mano sigue tendida, señor Casado.
Muchas gracias, señora presidenta. Señoría, ha hecho usted ha sido posar, delante de un espejo muy enfadado aparentemente, al lado de unas ovres conforme al IPC, compromiso de aumentar la dotación de becas en nuestro país, página 26 del programa electoral del Partido Socialista, compromiso de revalorizar las pensiones conforme al IPC, compromiso de revilirar que la Comisión Mixta de la Unión Europea. Usted ha comentado del Estado de alarma, es decir el mando único, pero también me pide más de un 10 % a los i